In [1]:
## 기본 library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
## model library
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from keras import models, layers, optimizers
## helper library
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from bayes_opt import BayesianOptimization

from sklearn.datasets import load_breast_cancer
from sklearn.cluster import KMeans

d = load_breast_cancer()

data = pd.DataFrame(d.data, columns=d.feature_names)
target = pd.DataFrame(d.target)



Using TensorFlow backend.


In [2]:
np.random.seed(0)

In [3]:
std = StandardScaler()
for c in data.columns:
    data[c] = std.fit_transform(data[c].values.reshape(-1, 1))
    
train_X, test_X, train_y, test_y = train_test_split(data, target, test_size=0.3, random_state=0, shuffle=True)

In [4]:
train_X = train_X.reset_index().drop('index', axis=1)
train_y = train_y.reset_index().drop('index', axis=1)
test_X = test_X.reset_index().drop('index', axis=1)
test_y = test_y.reset_index().drop('index', axis=1)

In [5]:
# km = KMeans(n_clusters=5).fit(train_X)
# cl = pd.get_dummies(km.labels_, prefix='kmean')
# train_X = pd.concat([train_X, cl], axis=1)

# # km = KMeans(n_clusters=5).fit(test_X)
# # cl = pd.get_dummies(km.labels_, prefix='kmean')
# # test_X = pd.concat([test_X, cl], axis=1)

# cl = pd.get_dummies(km.predict(test_X), prefix='kmean')
# test_X = pd.concat([test_X, cl], axis=1)

In [6]:
km = KMeans(n_clusters=5).fit(train_X)
tr_cl = pd.get_dummies(km.labels_, prefix='kmean')

te_cl = pd.get_dummies(km.predict(test_X), prefix='kmean')

In [7]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [8]:
lr = LogisticRegression(penalty='l2', random_state=0)

In [9]:
lr.fit(train_X, train_y)

c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
cross_val_score(lr, train_X, train_y, cv=kf)

c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

array([0.9875    , 0.975     , 0.95      , 0.98734177, 0.98734177])

In [11]:
accuracy_score(test_y, lr.predict(test_X))

0.9766081871345029

In [12]:
K.clear_session()
input_layer = layers.Input(shape=((train_X.shape[1:])))

l1 = layers.Dense(64 ,activation='relu')(input_layer)
l2 = layers.Dense(32, activation='relu')(l1)

out_layer = layers.Dense(1, activation='sigmoid')(l2)

model = models.Model(input_layer, out_layer)

model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['acc'])

In [13]:
es = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

In [14]:
model.fit(train_X, train_y, 
         epochs=300,
         validation_split=0.3,
         callbacks=[es]
         )

Train on 278 samples, validate on 120 samples
Epoch 1/300
278/278 [==============================] - 0s 1ms/step - loss: 0.5774 - acc: 0.8381 - val_loss: 0.4385 - val_acc: 0.9000
Epoch 2/300
278/278 [==============================] - 0s 104us/step - loss: 0.3573 - acc: 0.9640 - val_loss: 0.3034 - val_acc: 0.9583
Epoch 3/300
278/278 [==============================] - 0s 108us/step - loss: 0.2489 - acc: 0.9712 - val_loss: 0.2204 - val_acc: 0.9667
Epoch 4/300
278/278 [==============================] - 0s 97us/step - loss: 0.1810 - acc: 0.9676 - val_loss: 0.1683 - val_acc: 0.9583
Epoch 5/300
278/278 [==============================] - 0s 108us/step - loss: 0.1443 - acc: 0.9712 - val_loss: 0.1349 - val_acc: 0.9667
Epoch 6/300
278/278 [==============================] - 0s 108us/step - loss: 0.1176 - acc: 0.9748 - val_loss: 0.1161 - val_acc: 0.9750
Epoch 7/300
278/278 [==============================] - 0s 108us/step - loss: 0.1025 - acc: 0.9784 - val_loss: 0.1033 - val_acc: 0.9750
Epoch 8/300


Epoch 61/300
278/278 [==============================] - 0s 108us/step - loss: 0.0104 - acc: 0.9964 - val_loss: 0.0616 - val_acc: 0.9833
Epoch 62/300
278/278 [==============================] - 0s 115us/step - loss: 0.0103 - acc: 0.9964 - val_loss: 0.0608 - val_acc: 0.9833
Epoch 63/300
278/278 [==============================] - 0s 108us/step - loss: 0.0097 - acc: 0.9964 - val_loss: 0.0604 - val_acc: 0.9833
Epoch 64/300
278/278 [==============================] - 0s 104us/step - loss: 0.0094 - acc: 0.9964 - val_loss: 0.0603 - val_acc: 0.9833
Epoch 65/300
278/278 [==============================] - 0s 104us/step - loss: 0.0092 - acc: 0.9964 - val_loss: 0.0595 - val_acc: 0.9833
Epoch 66/300
278/278 [==============================] - 0s 108us/step - loss: 0.0090 - acc: 0.9964 - val_loss: 0.0585 - val_acc: 0.9833
Epoch 67/300
278/278 [==============================] - 0s 100us/step - loss: 0.0085 - acc: 0.9964 - val_loss: 0.0588 - val_acc: 0.9833
Epoch 68/300
278/278 [==========================

In [15]:
model.evaluate(test_X, test_y)

171/171 [==============================] - 0s 61us/step


[0.16014399441988453, 0.9649122953414917]

In [16]:
svc = SVC(kernel='rbf', random_state=0).fit(train_X, train_y)

c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
np.mean(cross_val_score(svc, train_X, train_y, cv=10))

c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

0.9800000000000001